## 导入库

## 1. 数据管理模块

In [21]:
import pandas as pd
import requests
from datetime import datetime

class DataManager:
    def __init__(self):
        self.data_path = "data/"
        self.license = "YOUR_FREE_LICENSE"  # **替换为麦蕊申请的证书**
        
    def load_data(self, symbol, start_date, end_date):
        # **麦蕊API调用示例（历史日线数据）**
        url = f"http://api.mairui.club/hsdayk/{symbol}/{self.license}"
        params = {
            "start": datetime.strptime(start_date, "%Y%m%d").strftime("%Y-%m-%d"),
            "end": datetime.strptime(end_date, "%Y%m%d").strftime("%Y-%m-%d")
        }
        
        # **请求并转换数据格式**
        response = requests.get(url, params=params)
        raw_data = response.json()['data']
        
        # **字段映射与类型转换**
        df = pd.DataFrame(raw_data)
        df = df.rename(columns={
            "date": "trade_date", 
            "close": "close",
            "vol": "vol"
        })
        df['trade_date'] = pd.to_datetime(df['trade_date'])
        df = df.sort_values('trade_date')  # 确保时间升序
        
        # 缓存到本地（保留原有逻辑）
        df.to_csv(f"{self.data_path}{symbol}.csv", index=False)
        return df

## 2. 策略开发模块
* 信号生成器：技术指标（如均线突破、RSI）
* 风险管理：止损/止盈、仓位控制
* 交易执行：订单类型（市价单、限价单）

In [12]:
class Strategy:
    def __init__(self, short_window=5, long_window=20):
        self.short_window = short_window
        self.long_window = long_window
        self.position = 0  # 持仓状态
    
    def generate_signal(self, data):
        data['short_ma'] = data['close'].rolling(self.short_window).mean()
        data['long_ma'] = data['close'].rolling(self.long_window).mean()
        # 金叉买入，死叉卖出
        if data['short_ma'].iloc[-1] > data['long_ma'].iloc[-1]:
            return 'BUY'
        elif data['short_ma'].iloc[-1] < data['long_ma'].iloc[-1]:
            return 'SELL'
        else:
            return 'HOLD'

## 3. 回测引擎模块
* 功能：模拟历史交易并评估策略表现
* __核心指标__：年化收益率、夏普比率、最大回撤、胜率

In [13]:
class Backtester:
    def __init__(self, initial_capital=100000):
        self.initial_capital = initial_capital
        self.portfolio = {'cash': initial_capital, 'shares': 0}
    
    def run_backtest(self, data, strategy):
        returns = []
        for i in range(len(data)):
            signal = strategy.generate_signal(data.iloc[:i+1])
            price = data.iloc[i]['close']
            if signal == 'BUY' and self.portfolio['cash'] > 0:
                self.portfolio['shares'] = self.portfolio['cash'] // price
                self.portfolio['cash'] -= self.portfolio['shares'] * price
            elif signal == 'SELL' and self.portfolio['shares'] > 0:
                self.portfolio['cash'] += self.portfolio['shares'] * price
                self.portfolio['shares'] = 0
            # 记录每日净值
            total_value = self.portfolio['cash'] + self.portfolio['shares'] * price
            returns.append(total_value / self.initial_capital - 1)
        return returns

## 4. 可视化模块
* __工具__ :Matplotlib/Seaborn 或交互式工具

In [24]:
import matplotlib.pyplot as plt
from IPython.display import display, clear_output  # Jupyter支持

def realtime_monitor(symbol, license):
    # **麦蕊实时行情接入示例**
    url = f"http://api.mairui.club/hsrl/ssjy/{symbol}/{license}"
    while True:
        data = requests.get(url).json()
        clear_output(wait=True)
        plt.figure(figsize=(8,4))
        plt.title(f"{symbol} 实时行情 | 最新价: {data['last_price']}")
        plt.bar(['开盘','最高','最低','最新'], 
                [data['open'], data['high'], data['low'], data['last_price']])
        plt.show()
        time.sleep(10)  # 每10秒刷新

## 5. 风险管理与绩效评估

In [16]:
import numpy as np

def calculate_performance(returns):
    annual_return = np.mean(returns) * 252
    volatility = np.std(returns) * np.sqrt(252)
    sharpe_ratio = annual_return / volatility
    max_drawdown = (np.maximum.accumulate(returns) - returns).max()
    return {
        "年化收益率": annual_return,
        "夏普比率": sharpe_ratio,
        "最大回撤": max_drawdown
    }

In [25]:
def validate_data(df):
    # **检查麦蕊数据完整性**
    required_columns = ['open', 'high', 'low', 'close', 'vol']
    missing = set(required_columns) - set(df.columns)
    if missing:
        raise ValueError(f"数据缺失关键字段: {missing}")
    
    # **检查异常值（如涨跌幅超过20%）**
    df['pct_chg'] = df['close'].pct_change()
    abnormal = df[abs(df['pct_chg']) > 0.2]
    if not abnormal.empty:
        print(f"警告：发现{len(abnormal)}条异常波动数据")

## 运行

In [26]:
if __name__ == "__main__":
    # **初始化使用麦蕊数据**
    dm = DataManager()
    data = dm.load_data("000001", "20230101", "20231231")  # 注意符号格式差异
    
    # 后续流程保持不变
    strategy = Strategy()
    backtester = Backtester()
    returns = backtester.run_backtest(data, strategy)
    
    # **可视化增加实时监控（可选）**
    # 在Jupyter中运行：实时监控
    # realtime_monitor("000001", dm.license)  
    
    performance = calculate_performance(returns)
    visualize_results(returns, data['close']/data['close'].iloc[0]-1)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)